# Part B2: Softmax Properties - Temperature & Numerical Stability

## Introduction

Welcome to **Part B2** of the Softmax Series!

**StatQuest Style**: "Now let's explore the cool properties of Softmax... clearly!"

### What We'll Cover

1. Temperature parameter (controlling sharpness)
2. Numerical stability trick (avoiding overflow)
3. Confidence vs uncertainty
4. Edge cases and practical considerations

### Learning Objectives

By the end of this notebook, you will:
- Understand the temperature parameter
- Implement numerically stable softmax
- Interpret confidence levels
- Handle edge cases properly

**Let's dive in!**

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print('✓ Libraries imported')
print('✓ Ready to explore Softmax properties!')

✓ Libraries imported
✓ Ready to explore Softmax properties!


---

# Part 1: Temperature Parameter

## Softmax with Temperature

```
┌─────────────────────────────────────────────────────────────┐
│         SOFTMAX WITH TEMPERATURE                            │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Standard Softmax:                                         │
│  ┌──────────────────────────────────────┐                 │
│  │         e^(x_i)                       │                 │
│  │  S_i = ─────────                      │                 │
│  │        Σ e^(x_j)                      │                 │
│  └──────────────────────────────────────┘                 │
│                                                             │
│  Softmax with Temperature T:                               │
│  ┌──────────────────────────────────────┐                 │
│  │         e^(x_i/T)                     │                 │
│  │  S_i = ───────────                    │                 │
│  │        Σ e^(x_j/T)                    │                 │
│  └──────────────────────────────────────┘                 │
│                                                             │
│  Temperature Effects:                                      │
│                                                             │
│  T → 0 (Low Temperature):                                  │
│  • Sharp distribution                                      │
│  • Confident predictions                                   │
│  • Approaches one-hot encoding                             │
│  • Like ArgMax                                             │
│                                                             │
│  T = 1 (Standard):                                         │
│  • Normal softmax                                          │
│  • Balanced                                                │
│                                                             │
│  T → ∞ (High Temperature):                                 │
│  • Soft distribution                                       │
│  • Uncertain predictions                                   │
│  • Approaches uniform distribution                         │
│  • All classes equally likely                              │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [2]:
# Implement softmax with temperature
def softmax_temperature(logits, temperature=1.0):
    """
    Calculate softmax with temperature parameter.
    
    Args:
        logits: Raw network outputs
        temperature: Temperature parameter (default=1.0)
    
    Returns:
        Probability distribution
    """
    scaled_logits = logits / temperature
    exp_logits = np.exp(scaled_logits)
    return exp_logits / np.sum(exp_logits)

# Test with different temperatures
logits = np.array([2.0, 1.0, 0.1])
classes = ['Cat', 'Dog', 'Mouse']
temperatures = [0.5, 1.0, 2.0, 5.0]

print('SOFTMAX WITH DIFFERENT TEMPERATURES')
print('='*70)
print(f'Logits: {logits}\n')

for T in temperatures:
    probs = softmax_temperature(logits, T)
    print(f'Temperature T = {T}:')
    for cls, p in zip(classes, probs):
        print(f'  {cls:<10} {p:.4f} ({p*100:>5.1f}%)')
    print()

print('='*70)
print('OBSERVATION:')
print('  Low T → Sharp (confident)')
print('  High T → Soft (uncertain)')

SOFTMAX WITH DIFFERENT TEMPERATURES
Logits: [2.  1.  0.1]

Temperature T = 0.5:
  Cat        0.8638 ( 86.4%)
  Dog        0.1169 ( 11.7%)
  Mouse      0.0193 (  1.9%)

Temperature T = 1.0:
  Cat        0.6590 ( 65.9%)
  Dog        0.2424 ( 24.2%)
  Mouse      0.0986 (  9.9%)

Temperature T = 2.0:
  Cat        0.5017 ( 50.2%)
  Dog        0.3043 ( 30.4%)
  Mouse      0.1940 ( 19.4%)

Temperature T = 5.0:
  Cat        0.3996 ( 40.0%)
  Dog        0.3272 ( 32.7%)
  Mouse      0.2733 ( 27.3%)

OBSERVATION:
  Low T → Sharp (confident)
  High T → Soft (uncertain)


---

# Part 2: Numerical Stability

## The Overflow Problem

```
┌─────────────────────────────────────────────────────────────┐
│         NUMERICAL STABILITY PROBLEM                         │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Problem: Large logits cause overflow                      │
│                                                             │
│  Example:                                                  │
│  ┌──────────────────────────────────────┐                 │
│  │  Logits: [1000, 1001, 999]           │                 │
│  │                                       │                 │
│  │  e^1000 = HUGE NUMBER!                │                 │
│  │  → Overflow error                     │                 │
│  │  → NaN or Inf                         │                 │
│  └──────────────────────────────────────┘                 │
│                                                             │
│  Solution: Subtract maximum before exp                     │
│                                                             │
│  Stable Softmax:                                           │
│  ┌──────────────────────────────────────┐                 │
│  │  1. Find max: M = max(x)             │                 │
│  │  2. Subtract: x' = x - M             │                 │
│  │  3. Apply softmax to x'              │                 │
│  └──────────────────────────────────────┘                 │
│                                                             │
│  Why this works:                                           │
│  ┌──────────────────────────────────────┐                 │
│  │  e^(x_i - M)     e^x_i / e^M         │                 │
│  │  ─────────── = ───────────────        │                 │
│  │  Σ e^(x_j - M)   Σ e^x_j / e^M       │                 │
│  │                                       │                 │
│  │  e^M cancels out!                     │                 │
│  │  Result is mathematically identical   │                 │
│  └──────────────────────────────────────┘                 │
│                                                             │
│  After subtracting max:                                    │
│  • Largest value becomes 0                                 │
│  • All others are negative                                 │
│  • e^0 = 1 (safe!)                                         │
│  • e^(negative) < 1 (safe!)                                │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [3]:
# Implement numerically stable softmax
def softmax_stable(logits):
    """
    Numerically stable softmax implementation.
    
    Args:
        logits: Raw network outputs
    
    Returns:
        Probability distribution
    """
    # Step 1: Subtract maximum for numerical stability
    logits_shifted = logits - np.max(logits)
    
    # Step 2: Exponentiate
    exp_logits = np.exp(logits_shifted)
    
    # Step 3: Normalize
    return exp_logits / np.sum(exp_logits)

# Compare unstable vs stable
print('NUMERICAL STABILITY DEMONSTRATION')
print('='*70)

# Small logits (both work)
small_logits = np.array([1.0, 2.0, 3.0])
print('Small logits:', small_logits)
print('Unstable softmax:', np.exp(small_logits) / np.sum(np.exp(small_logits)))
print('Stable softmax:  ', softmax_stable(small_logits))
print()

# Large logits (unstable fails)
large_logits = np.array([1000.0, 1001.0, 999.0])
print('Large logits:', large_logits)
print('Unstable softmax:', np.exp(large_logits) / np.sum(np.exp(large_logits)))
print('Stable softmax:  ', softmax_stable(large_logits))
print()

print('='*70)
print('CONCLUSION:')
print('  Always use the stable version in production!')
print('  Subtract max before exponentiating.')

NUMERICAL STABILITY DEMONSTRATION
Small logits: [1. 2. 3.]
Unstable softmax: [0.09003057 0.24472847 0.66524096]
Stable softmax:   [0.09003057 0.24472847 0.66524096]

Large logits: [1000. 1001.  999.]
Unstable softmax: [nan nan nan]
Stable softmax:   [0.24472847 0.66524096 0.09003057]

CONCLUSION:
  Always use the stable version in production!
  Subtract max before exponentiating.


/tmp/ipython-input-417633057.py:35: RuntimeWarning: overflow encountered in exp
  print('Unstable softmax:', np.exp(large_logits) / np.sum(np.exp(large_logits)))
/tmp/ipython-input-417633057.py:35: RuntimeWarning: invalid value encountered in divide
  print('Unstable softmax:', np.exp(large_logits) / np.sum(np.exp(large_logits)))


---

# Summary

## Key Takeaways

1. **Temperature Parameter**:
   - Controls distribution sharpness
   - Low T → confident (sharp)
   - High T → uncertain (soft)
   - Used in knowledge distillation

2. **Numerical Stability**:
   - Subtract max before exp
   - Prevents overflow
   - Mathematically equivalent
   - Always use in production

3. **Best Practices**:
   - Always implement stable version
   - Consider temperature for specific tasks
   - Monitor for NaN/Inf values

## What's Next?

In **Part C1**, we'll:
- Derive the softmax derivative (case i=j)
- Use the quotient rule
- Get the beautiful result: S_i(1 - S_i)
- Understand the geometric interpretation

**Double Bam!** 💥💥